In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as mplStyle
mplStyle.use('plot_style.mpl')
import h5py
from scipy.optimize import curve_fit
from scipy import signal
from matplotlib import cm
from scipy import interpolate
from scipy import ndimage
from scipy import special
import copy
import control as ct

In [ ]:
data_filename = '../data/KSTAR_detach_ctrl_data.h5'
psi_data_filename = '../data/KSTAR_detach_ctrl_Psi_data.h5'
# data_filename = '/Users/gupta/research/conferences/KSTAR_Fall_2024/experiments/data_bank/KSTAR_detach_ctrl_data.h5'

def clip(data, min_sig, max_sig):
    return np.maximum(np.minimum(data, max_sig), min_sig)

def clip_flow(h5, sn, port, species):
    root = 'PVB' + port.upper() + species.upper()
    clipped_cmd = clip(h5[sn]['PCS_KSTAR']['\\GVS' + root]['data'][:] * h5[sn]['PCS_KSTAR']['\\GVI' + root + 'ON']['data'][:],
                       h5[sn]['PCS_KSTAR']['\\GVT' + root + 'MIN']['data'][:],
                       h5[sn]['PCS_KSTAR']['\\GVT' + root + 'MAX']['data'][:])
    return h5[sn]['PCS_KSTAR']['\\GVS' + root]['dim0'][:], clipped_cmd

def gas_flow(h5, sn, port, species):
    prt = {'D': 'DH', 'L': 'LP'}[port.upper()]
    sp = {'D2': 'D', 'N2': 'N', 'NE': 'E', 'AR': 'A', 'KR': 'K'}[species.upper()]
    pn = f'\\{prt}_{sp}FLOW_OUT:FOO'
    return h5[sn]['KSTAR'][pn]['dim0'][:], h5[sn]['KSTAR'][pn]['data'][:]

def cum_flow(h5, sn, port, species):
    tt, flow_rate = gas_flow(h5, sn, port, species)
    cum_flow = np.cumsum(flow_rate[:-1] * (tt[1:] - tt[:-1]))
    return tt[:-1], cum_flow

def ch(h5, sn, tree, pn):
    return h5[sn][tree][pn]['dim0'][:], h5[sn][tree][pn]['data'][:]

LPz = np.array([-1.275, -1.2375,  -1.225, -1.2125, -1.175, -1.1625])
LPr = 1.625 * np.ones_like(LPz)
LPlabels = ['OD5', 'OD8', 'OD9', 'OD11', 'OD13', 'OD15']

def add_subplot_ind(ax, axr, indstr, **kwargs):
    axxlims = ax.get_xlim()
    axylims = ax.get_ylim()
    if ax.xaxis.get_scale() == 'linear':
        xpos = axxlims[0] * (1 - axr) + axxlims[1] * axr
    elif ax.xaxis.get_scale() == 'log':
        xpos = 10**(np.log10(axxlims[0]) * (1 - axr) + np.log10(axxlims[1]) * axr)
    if ax.yaxis.get_scale() == 'linear':
        ypos = axylims[0] * axr + axylims[1] * (1 - axr)
    elif ax.yaxis.get_scale() == 'log':
        ypos = 10**(np.log10(axylims[0]) * axr + np.log10(axylims[1]) * (1 - axr))
    ax.text(xpos, ypos, indstr, horizontalalignment='left', verticalalignment='top',
            weight='bold', **kwargs)

def get_sep(Ri, Zi, psini, wall, z_upper_lim=0.8):
    fig, ax = plt.subplots()
    null_surface = ax.contour(Ri, Zi, psini, [1.0]).get_paths()[0].vertices
    plt.close(fig)

    sep_r = np.array([])
    sep_z = np.array([])
    for ii in range(null_surface.shape[0]):
        r, z = null_surface[ii, :]
        noi = 0
        for panel_ind in range(wall.shape[0]):
            r1, z1 = wall[panel_ind, :]
            r2, z2 = wall[(panel_ind + 1) % wall.shape[0], :]
            if (z1 < z) != (z2 < z) and r < r1 and r < r2 and z < z_upper_lim:
                noi += 1
        if noi % 2 == 1:
            sep_r = np.append(sep_r, r)
            sep_z = np.append(sep_z, z)
    return sep_r, sep_z

In [ ]:
sn = '35851'
pick_time = 8.0
zoom_fac = 10

axxlims = 1.15, 2.4
axylims = -1.4, 1.45
iaxxlims = 1.615, 1.64
iaxylims = -1.3, -1.15
baxxlims = 0.0, 18.0

axr=iaxr=baxr=0.01
fig, ax = plt.subplots(1, 1, figsize=(3.5, 7.5))

with h5py.File(psi_data_filename, 'r') as h5:
    tt = h5[sn]['EFIT01']['\\PSIRZ']['dim2'][:] * 1e-3
    ttind = np.argmin(np.abs(tt - pick_time))
    R = h5[sn]['EFIT01']['\\PSIRZ']['dim0'][:]
    Z = h5[sn]['EFIT01']['\\PSIRZ']['dim1'][:]
    psi = h5[sn]['EFIT01']['\\PSIRZ']['data'][ttind, :, :]
    a = h5[sn]['EFIT01']['\\SSIMAG']['data'][ttind]
    b = h5[sn]['EFIT01']['\\SSIBRY']['data'][ttind]
    psin = (psi - a) / (b - a)
    Ri = ndimage.zoom(R, zoom_fac)
    Zi = ndimage.zoom(Z, zoom_fac)
    psini = ndimage.zoom(psin, zoom_fac)
    wall = h5[sn]['EFIT01']['\\LIM']['data'][:]

# Create the inset axes
inset_ax = ax.inset_axes([0.37, 0.35, 0.31, 0.3]) # [x, y, width, height]

sep = get_sep(Ri, Zi, psini, wall)
for sax in [ax, inset_ax]:
    sax.plot(sep[0], sep[1], color='tab:blue')
    sax.plot(wall[:, 0], wall[:, 1], color='black')
    sax.scatter(LPr, LPz, 25, color='red')

ax.set_xlim(axxlims)
ax.set_ylim(axylims)
ax.set_aspect('equal')
ax.set_xlabel('R / m')
ax.set_ylabel('Z / m')

inset_ax.set_xlim(iaxxlims) # Set the limits for the zoomed-in region
inset_ax.set_ylim(iaxylims)
inset_ax.set_yticks(LPz)

for ii, od in enumerate(LPlabels):
    inset_ax.text(LPr[ii] + 0.001, LPz[ii], od, verticalalignment='center')

# Draw a rectangle on the main plot to indicate the zoomed-in region
ax.indicate_inset_zoom(inset_ax, edgecolor="black")

ax.text(axxlims[1]-0.05, axylims[1] - 0.05, f'KSTAR #{sn}\nt={pick_time}',
        horizontalalignment='right', verticalalignment='top')

with h5py.File(data_filename, 'r') as h5:
    tt = h5[sn]['PCS_KSTAR']['\\DVSIP']['dim0'][:]
    ip = -h5[sn]['PCS_KSTAR']['\\DVSIP']['data'][:]
    beta = h5[sn]['PCS_KSTAR']['\\EFSBETAN']['data'][:]

bottom_ax = ax.inset_axes([0.0, -0.18, 1.0, 0.1])
bottom_ax.plot(tt, ip, color='tab:red')
bottom_ax.plot(tt, beta, color='tab:green')
bottom_ax.set_xlim(baxxlims)
bottom_ax.text(6.0, max(ip), 'I' + r'$_{p} / $' + ' kA',
               verticalalignment='bottom', color='tab:red')
bottom_ax.text(12.0, max(beta)*0.8, r'$\beta_{n}$',
               verticalalignment='top', color='tab:green')
bottom_ax.set_xlabel('Time / s')

add_subplot_ind(ax, axr, '(a)')
add_subplot_ind(inset_ax, iaxr, '(b)')
add_subplot_ind(bottom_ax, baxr, '(c)')

# fig.savefig('../figures/RefShot.pdf', bbox_inches='tight')

In [ ]:

lporder = 4
fcut = 20
LPFsos = signal.butter(lporder, fcut, 'low', output='sos', fs=1000)
ms = 1
ds = 25
use_LP = [1, 2]

ary = 0.05
arl = 0.01
arhl = 0.005
arhw = 0.01
arw = 0.001

starttime = 7
endtime = 16

def strike_profile(z, A, mu, sig, offset):
    return A * np.exp(-((z- mu)/sig)**2) + offset


def eich_profile(z, A, S, l, offset, mu):
    return A * np.exp((S/l/2)**2 - z/l + mu/l) * special.erfc(S/l/2 - z/S + mu/S) + offset

trial_p = [0.3, 0.01, 0.005, 0.1, -0.005]



fig, axs = plt.subplots(1, 1, figsize=(3.5, 5),
                        sharex=True, sharey=True,
                        gridspec_kw={'hspace': 0.05})

with h5py.File(data_filename, 'r') as h5:
    sn = '35851'
    ax = axs
    tt = h5[sn]['PCS_KSTAR']['\\PCLPEP51']['dim0'][:]
    startind = np.argmin(np.abs(tt - starttime))
    endind = np.argmin(np.abs(tt - endtime))
    LPdata = np.ones((len(LPz), len(tt))) * np.nan
    for ii in range(len(LPz)):
        LPdata[ii, :] = signal.sosfiltfilt(LPFsos, h5[sn]['PCS_KSTAR']['\\PCLPEP5' + str(ii + 1)]['data'][:])
    Zosp = signal.sosfiltfilt(LPFsos, h5[sn]['EFIT01']['\\ZVSOUT']['data'][:])
    z_all = []
    i_all = []
    for ii in range(len(LPz)):
        z = LPz[ii] - Zosp[startind:endind:ds]
        isat = LPdata[ii, startind:endind:ds]
        ax.scatter(z, isat, ms, label=LPlabels[ii])
        if ii in use_LP:
            z_all += list(z)
            i_all += list(isat)
    z_all = np.array(z_all)
    i_all = np.array(i_all)
    fitz = np.linspace(*ax.get_xlim(), 100)
    ax.plot(fitz, eich_profile(fitz, *trial_p))
    popt, pcov = curve_fit(eich_profile, z_all, i_all,
                           bounds=([0.1, 0.001, 1e-4, 0.05, -0.01],
                                   [10.0, 0.02, 0.1, 0.15, -0.00]))
    print(popt)
    ax.plot(fitz, eich_profile(fitz, *popt))

In [ ]:


lporder = 4
fcut = 20
LPFsos = signal.butter(lporder, fcut, 'low', output='sos', fs=1000)
ms = 1
ds = 25
use_LP = [1, 2]

ary = 0.05
arl = 0.01
arhl = 0.005
arhw = 0.01
arw = 0.001

starttime = 7
endtime = 16

def strike_profile(z, A, mu, sig, offset):
    return A * np.exp(-((z- mu)/sig)**2) + offset

def eich_profile(z, A, mu, sig, offset, P):
    return A * np.exp(((z- mu)/sig)) * special.erfc(-z*P) + offset

trial_p = [0.15, -0.005, 0.006, 0.1, 20]

fig, axs = plt.subplots(2, 1, figsize=(3.5, 5),
                        sharex=True, sharey=True,
                        gridspec_kw={'hspace': 0.05})

with h5py.File(data_filename, 'r') as h5:
    sn = '35851'
    ax = axs[0]
    tt = h5[sn]['PCS_KSTAR']['\\PCLPEP51']['dim0'][:]
    startind = np.argmin(np.abs(tt - starttime))
    endind = np.argmin(np.abs(tt - endtime))
    LPdata = np.ones((len(LPz), len(tt))) * np.nan
    for ii in range(len(LPz)):
        LPdata[ii, :] = signal.sosfiltfilt(LPFsos, h5[sn]['PCS_KSTAR']['\\PCLPEP5' + str(ii + 1)]['data'][:])
    Zosp = signal.sosfiltfilt(LPFsos, h5[sn]['EFIT01']['\\ZVSOUT']['data'][:])
    z_all = []
    i_all = []
    for ii in range(len(LPz)):
        z = LPz[ii] - Zosp[startind:endind:ds]
        isat = LPdata[ii, startind:endind:ds]
        ax.scatter(z, isat, ms, label=LPlabels[ii])
    
    hwloc = [-0.01, 0.002]
    ax.vlines(hwloc, *ax.get_ylim(), color="black", ls=':')
    ax.arrow(x=hwloc[0] - arl - arhl, y=ary, dx=arl, dy=0, width=arw,
             head_width=arhw, head_length=arhl, fc='k')
    ax.arrow(x=hwloc[1] + arl + arhl, y=ary, dx=-arl, dy=0, width=arw,
             head_width=arhw, head_length=arhl, fc='k')
    ax.text(hwloc[1] + arl + arhl, ary, r"$\approx$" + f" {round((hwloc[1] - hwloc[0])*1e3)} mm")
    ax.set_ylabel("I$_{sat}$ / A")
    

    sn = '35855'
    ax = axs[1]
    tt = h5[sn]['PCS_KSTAR']['\\PCLPEP51']['dim0'][:]
    startind = np.argmin(np.abs(tt - starttime))
    endind = np.argmin(np.abs(tt - endtime))
    LPdata = np.ones((len(LPz), len(tt))) * np.nan
    for ii in range(len(LPz)):
        LPdata[ii, :] = signal.sosfiltfilt(LPFsos, h5[sn]['PCS_KSTAR']['\\PCLPEP5' + str(ii + 1)]['data'][:])
    Zosp = signal.sosfiltfilt(LPFsos, h5[sn]['EFIT01']['\\ZVSOUT']['data'][:])
    z_all = []
    i_all = []
    for ii in range(len(LPz)):
        z = LPz[ii] - Zosp[startind:endind:ds]
        isat = LPdata[ii, startind:endind:ds]
        ax.scatter(z, isat, ms, label=LPlabels[ii])
    hwloc = [-0.006, 0.006]
    ax.vlines(hwloc, *ax.get_ylim(), color="black", ls=':')
    ax.arrow(x=hwloc[0] - arl - arhl, y=ary, dx=arl, dy=0, width=arw,
             head_width=arhw, head_length=arhl, fc='k')
    ax.arrow(x=hwloc[1] + arl + arhl, y=ary, dx=-arl, dy=0, width=arw,
             head_width=arhw, head_length=arhl, fc='k')
    ax.text(hwloc[1] + arl + arhl, ary, r"$\approx$" + f" {round((hwloc[1] - hwloc[0])*1e3)} mm")
    ax.set_ylabel("I$_{sat}$ / A")

axs[0].legend()
axs[1].set_xlabel('Z - Z$_{OSP}$ / m')

add_subplot_ind(axs[0], 0.01, f"(a) KSTAR #35851")
add_subplot_ind(axs[1], 0.01, f"(b) KSTAR #35855")
# fig.savefig('../figures/StrikePointWidth.pdf', bbox_inches='tight')

In [ ]:


lporder = 4
fcut = 20
LPFsos = signal.butter(lporder, fcut, 'low', output='sos', fs=1000)
ms = 5
ds = 25
use_LP = [1, 2]

ary = 0.175
arl = 10
arhl = 5
arhw = 0.005
arw = 0.0005

starttime = 7
endtime = 16

def strike_profile(z, A, mu, sig, offset):
    return A * np.exp(-((z- mu)/sig)**2) + offset

def eich_profile(z, A, mu, sig, offset, P):
    return A * np.exp(((z- mu)/sig)) * special.erfc(-z*P) + offset

trial_p = [0.15, -0.005, 0.006, 0.1, 20]

fig, ax = plt.subplots(1, 1, figsize=(3.5, 3.5))

with h5py.File(data_filename, 'r') as h5:
    sn = '35851'
    tt = h5[sn]['PCS_KSTAR']['\\PCLPEP51']['dim0'][:]
    startind = np.argmin(np.abs(tt - starttime))
    endind = np.argmin(np.abs(tt - endtime))
    LPdata = np.ones((len(LPz), len(tt))) * np.nan
    for ii in range(len(LPz)):
        LPdata[ii, :] = signal.sosfiltfilt(LPFsos, h5[sn]['PCS_KSTAR']['\\PCLPEP5' + str(ii + 1)]['data'][:])
    Zosp = signal.sosfiltfilt(LPFsos, h5[sn]['EFIT01']['\\ZVSOUT']['data'][:])
    z_all = []
    i_all = []
    for ii in range(len(LPz)):
        z = (LPz[ii] - Zosp[startind:endind:ds]) * 1e3
        isat = LPdata[ii, startind:endind:ds]
        ax.scatter(z, isat, ms, label=LPlabels[ii])
    
    hwloc = [-11, 3]
    ax.vlines(hwloc, *ax.get_ylim(), color="black", ls=':')
    ax.arrow(x=hwloc[0] - arl - arhl, y=ary, dx=arl, dy=0, width=arw,
             head_width=arhw, head_length=arhl, fc='k')
    ax.arrow(x=hwloc[1] + arl + arhl, y=ary, dx=-arl, dy=0, width=arw,
             head_width=arhw, head_length=arhl, fc='k')
    ax.text(hwloc[1] + arl + arhl, ary, r"$\approx$" + f" {round((hwloc[1] - hwloc[0]))} mm")
    ax.set_ylabel("I$_{sat}$ / A")
ax.set_ylim(0.05, 0.26)
ax.set_xlabel('Z - Z$_{OSP}$ / mm')
ax.legend()
add_subplot_ind(ax, 0.01, f"KSTAR #{sn}")

# fig.savefig('../figures/StrikePointWidth.pdf', bbox_inches='tight')

In [ ]:
sn = '35851'

lporder = 4
fcut = 20
LPFsos = signal.butter(lporder, fcut, 'low', output='sos', fs=1000)

starttime = 7
endtime = 16


fig, ax = plt.subplots(1, 1, figsize=(3.5, 3.5))

with h5py.File(data_filename, 'r') as h5:
    tt = h5[sn]['PCS_KSTAR']['\\PCLPEP51']['dim0'][:]
    startind = np.argmin(np.abs(tt - starttime))
    endind = np.argmin(np.abs(tt - endtime))
    LPdata = np.ones((len(LPz), len(tt))) * np.nan
    for ii in range(len(LPz)):
        LPdata[ii, :] = signal.sosfiltfilt(LPFsos, h5[sn]['PCS_KSTAR']['\\PCLPEP5' + str(ii + 1)]['data'][:])
    Rosp = signal.sosfiltfilt(LPFsos, h5[sn]['EFIT01']['\\ZVSOUT']['data'][:])
    ax.plot(tt[startind:endind], LPdata[0, startind:endind])


## System Identification

In [ ]:
def fit_plant(ts, sig, command, tlims=None, pre_ext=5, y_zero_offset=0,
              bounds=([-np.inf, 0, 0], [np.inf, 10, 10]),
              pade_order=10):
    if tlims is None:
        indStart = 0
        indEnd = len(ts)
    else:
        indStart = np.argmin(np.abs(ts - tlims[0]))
        indEnd = np.argmin(np.abs(ts - tlims[1]))

    com_cut = command[indStart:indEnd]
    sig_cut = sig[indStart:indEnd]

    if com_cut[0] == 0.0:
        first_non_zero_ind = np.nonzero(com_cut)[0][0]
        y_zero_offset = np.mean(sig_cut[:first_non_zero_ind])

    sig_cut = sig_cut - y_zero_offset
    
    ts_cut = ts[indStart:indEnd]
    dt = ts_cut[1] - ts_cut[0]
    # ts_pre = np.flip(np.arange(ts[indStart] - dt, ts[indStart] - pre_ext, -dt))
    # ts_ext = np.concatenate([ts_pre, ts])
    ts_ext = np.arange(ts[indStart] - pre_ext, ts[indEnd], dt)
    
    def plant_model(xx, K, tau, L):
        trial_plant_no_delay = ct.tf([K / tau], [1, 1 / tau])
        trial_plant_delay = ct.tf(*ct.pade(L, pade_order))
        trial_plant = ct.series(trial_plant_no_delay, trial_plant_delay)
        xx_pre = xx[0] * np.ones(len(ts_ext) - len(ts_cut))
        xx_ext = np.concatenate([xx_pre, xx])
        _, yy_ext = ct.forced_response(trial_plant, ts_ext, xx_ext, return_x=False)
        yy = yy_ext[len(xx_pre):]
        return yy

    popt, pcov = curve_fit(plant_model, com_cut, sig_cut, bounds=bounds)
    perr = np.sqrt(np.diag(pcov))
    fit_yy = plant_model(com_cut, *popt)

    def fitted_plant(tt, xx, K=popt[0], tau=popt[1], L=popt[2]):
        plant_no_delay = ct.tf([K / tau], [1, 1 / tau])
        plant_delay = ct.tf(*ct.pade(L, pade_order))
        plant = ct.series(plant_no_delay, plant_delay)
        _, yy = ct.forced_response(plant, tt, xx, return_x=False)
        return yy + y_zero_offset


    pmin = popt - perr
    pmax = popt + perr
    pstep = perr / 20
    retDict = {'K': popt[0], 'Kmin': pmin[0], 'Kmax': pmax[0], 'Kstep': pstep[0],
               'tau': popt[1], 'taumin': pmin[1], 'taumax': pmax[1], 'taustep': pstep[1],
               'L': popt[2], 'Lmin': pmin[2], 'Lmax': pmax[2], 'Lstep': pstep[2],
               'fit_tt': ts_cut, 'fit_yy': fit_yy + y_zero_offset,
               'popt': popt, 'perr': perr, 'pcov': pcov, 'fitted_plant': fitted_plant}
    return retDict

def fit_shot(sn, sigpn, compn, tlims=None, pre_ext=5, y_zero_offset=0,
             bounds=([-np.inf, 0, 0], [np.inf, 10, 10]),
             pade_order=10, sig_adhoc_fac=1.0):
    with h5py.File(data_filename, 'r') as h5:
        sigtt, sig = ch(h5, sn, 'PCS_KSTAR', sigpn)
        comptt, com = ch(h5, sn, 'PCS_KSTAR', compn)
    return fit_plant(sigtt, sig * sig_adhoc_fac, com, tlims=tlims, pre_ext=pre_ext,
                     y_zero_offset=y_zero_offset,
                     bounds=bounds,
                     pade_order=pade_order)

def loop_stability(plant_tau=0.726,
                   plant_K=0.323,
                   plant_L=0.139,
                   Kp = 10,
                   Ti = 1.2,
                   Td = 0.726,
                   smooth_tau=0.005):
    
    # Define plant
    plant_no_delay = ct.tf([plant_K / plant_tau], [1, 1 / plant_tau])
    plant_delay = ct.tf(*ct.pade(plant_L, 10))
    plant = ct.series(plant_no_delay, plant_delay)

    # Define PID with pre-smoothing

    PID_presmooth = ct.tf([1 / smooth_tau], [1, 1 / smooth_tau])
    PID_only = ct.tf([Kp * Td * Ti, Kp * Ti, Kp], [Ti, 0])

    PID = ct.series(PID_presmooth, PID_only)

    # Calculate transfer functions
    ff = np.logspace(-2, 2, 400)
    omg = 2 * np.pi * ff
    mag, ph, _ = ct.frequency_response(plant, omg)
    plant_TF = mag * np.exp(1j * ph)

    PID_mag, PID_ph, _ = ct.frequency_response(PID, omg)
    PID_TF = PID_mag * np.exp(1j * PID_ph)

    OLTF = plant_TF * PID_TF
    OLTF_mag = np.abs(OLTF)
    OLTF_ph = np.angle(OLTF, deg=True)

    CLTF = plant_TF / ( 1 + OLTF)

    UGF_ind = np.argmin(np.abs(OLTF_mag - 1.0))
    UGF = ff[UGF_ind]
    phase_margin = OLTF_ph[UGF_ind] + 180
    delay_margin = (phase_margin  * np.pi / 180) / (2 * np.pi * UGF)
    PSF_ind = np.argmin(np.abs(np.unwrap(OLTF_ph * np.pi / 180) + np.pi))
    PSF = ff[PSF_ind]
    gain_margin = 1 / OLTF_mag[PSF_ind]
    return ff, plant_TF, PID_TF, OLTF, CLTF, Kp, smooth_tau, Ti, Td, UGF, PSF, gain_margin, phase_margin, delay_margin


### Afrac SysID

In [ ]:
fit_vals = {}

fit_vals['35853'] = fit_shot('35853', '\\DVSAFRAC', '\\GVSPVBLN2',
                            tlims=(6.5, 14.75), pre_ext=0, y_zero_offset=5.0,
                            bounds=([-np.inf, 0, 0], [np.inf, 1, 1.1]), pade_order=20,
                            sig_adhoc_fac=0.5)

fit_vals['35853']


In [ ]:
sn = '35853'
LPz=LPz
lporder = 4
fcut = 20
vmin = 0.05
vmax = 0.25
starttime = 7.5
colormap = cm.plasma

LPFsos = signal.butter(lporder, fcut, 'low', output='sos', fs=1000)

fig, axs = plt.subplots(4, 1, sharex=True, gridspec_kw={'hspace': 0.1, 'height_ratios':[1.0, 1.0, 1.0, 0.5]},
                    figsize=(7.5, 4.5))
ax = axs[0]
axafrac = axs[1]
axN2 = axs[2]
axbeta = axs[3]
with h5py.File(data_filename, 'r') as h5:
    tt, afrac = ch(h5, sn, 'PCS_KSTAR', '\\DVSAFRAC')
    axafrac.plot(tt, afrac * 0.5)
    axafrac.plot(fit_vals[sn]['fit_tt'], fit_vals[sn]['fit_yy'],
                 label='SysID Fit', color='black', ls='--')

    axN2.plot(*clip_flow(h5, sn, 'L', 'N2'))
    caxN2 = axN2.twinx()
    caxN2.plot(*cum_flow(h5, sn, 'L', 'N2'), ls='--', color='tab:brown')

    axbeta.plot(h5[sn]['PCS_KSTAR']['\\EFSBETAN']['dim0'][:],
            h5[sn]['PCS_KSTAR']['\\EFSBETAN']['data'][:],)
    axip = axbeta.twinx()

    axip.plot(h5[sn]['PCS_KSTAR']['\\DVSIP']['dim0'][:],
                -h5[sn]['PCS_KSTAR']['\\DVSIP']['data'][:], ls='--', color='tab:brown')
    
    tt = h5[sn]['PCS_KSTAR']['\\PCLPEP51']['dim0'][:]
    LPdata = np.ones((len(LPz), len(tt))) * np.nan
    for ii in range(len(LPz)):
        LPdata[ii, :] = signal.sosfiltfilt(LPFsos, h5[sn]['PCS_KSTAR']['\\PCLPEP5' + str(ii + 1)]['data'][:])
    
    LPz_arr = np.linspace(min(LPz), max(LPz), 100)
    LPcs = interpolate.CubicSpline(LPz, LPdata, axis=0, bc_type='natural')
    LPplot = ax.pcolor(tt[::20], LPz_arr, LPcs(LPz_arr)[:, ::20], cmap=colormap, shading='nearest', vmin=vmin, vmax=vmax, label=None, edgecolor='face')

    ax.hlines(LPz, color='grey', ls='--', xmin=ax.get_xlim()[0], xmax=ax.get_xlim()[1], alpha=0.8)

    try:
        ax.plot(h5[sn]['EFIT01']['\\ZVSOUT']['dim0'][:],
                h5[sn]['EFIT01']['\\ZVSOUT']['data'][:],
                color='black',
                label='Z' + r'$_{Strike, out}$' + ' from EFIT')
    except BaseException:
        pass
    
    for ii in range(len(tt)):
        if tt[ii] > starttime and  -h5[sn]['PCS_KSTAR']['\\DVSIP']['data'][ii] < 0.4:
            endtime = tt[ii]
            # print(endtime)
            break

fitK = fit_vals[sn]['K'][()]
fitKerr = fit_vals[sn]['perr'][0][()]
fittau = fit_vals[sn]['tau'][()]
fittauerr = fit_vals[sn]['perr'][1][()]
fitL = fit_vals[sn]['L'][()]
fitLerr = fit_vals[sn]['perr'][2][()]
props = dict(boxstyle='round', facecolor='white', alpha=0.8)
axafrac.text(endtime-1.55, 0.3, f'K = {fitK:0.3f}' + r'$\pm$' + f'{fitKerr:0.3f}\n'
                + r'$\tau$' + f' = {fittau:0.2f}' + r'$\pm$' + f'{fittauerr:0.2f}' + ' s\n'
                + f'L = {fitL:0.3f}' + r'$\pm$' + f'{fitLerr:0.3f} s',
            bbox=props)

# with open('Afrac_SysID_values.tex', 'w') as f:
#     f.write(r'\newcommand{\AfracK}{' + f'K = {fitK:0.3f}' + r'$\pm$' + f'{fitKerr:0.3f}' + '}\n')
#     f.write(r'\newcommand{\AfracTau}{' + r'$\tau$' + f' = {fittau:0.2f}' + r'$\pm$' + f'{fittauerr:0.2f}' + 's}\n')
#     f.write(r'\newcommand{\AfracL}{' + f'L = {fitL:0.3f}' + r'$\pm$' + f'{fitLerr:0.3f}' + 's}\n')
    

caxl = fig.add_axes([0.91, 0.47, 0.015, 0.4])
fig.colorbar(LPplot, cax = caxl, label='LP Array I' + r'$_{sat}$' + ' / A')

ax.set_ylim(min(LPz)-0.002, max(LPz)+0.002)
axafrac.set_ylim(-0., 1.2)
# caxN2.set_ylim(-1, 6)
axbeta.set_ylim(0, 3.0)
axip.set_ylim(0, 0.75)

ax.set_ylabel('Z / m')
axafrac.set_ylabel('A' + '$_{frac}^*$')
axN2.set_ylabel(r'PVBL N$_2$' + '\nCmd / V')
caxN2.set_ylabel('Total Particles\n/1e19', color='tab:brown')
axbeta.set_ylabel(r'$\beta_N$')
axip.set_ylabel('I' + r'$_p$' + ' / MA', color='tab:brown')

ax.legend(
    # loc=(0.6, 0.05), 
    loc='lower right',
    framealpha=0.5)
axafrac.legend(ncols=2,
            #    loc=(0.6, 0.75),
               loc='upper right',
)

# axs[0].set_title(f'#{sn} Langmuir Probe Spatio-Temporal Data Low Passed by {2*lporder}th order Butterworth Filter at {fcut} Hz')

axs[-1].set_xlabel('Time / s')

axs[-1].set_xlim((starttime, endtime))

add_subplot_ind(ax, 0.01, '(a)')
add_subplot_ind(axafrac, 0.01, '(b)')
add_subplot_ind(axN2, 0.01, '(c)')
add_subplot_ind(axbeta, 0.01, '(d)')

ax.grid(False, 'both')
caxN2.yaxis.grid(False, which='minor')
axip.yaxis.grid(False, which='minor')

# fig.savefig(f'../figures/DetCtrl_2D_{sn}.pdf', bbox_inches='tight')

In [ ]:
AfracKp = -10
AfracTi=0.253
Afracsmooth_tau=0.05
ff, plant_TF, PID_TF, OLTF, CLTF, Kp, smooth_tau, Ti, Td, UGF, PSF, gain_margin, phase_margin, delay_margin = loop_stability(
    plant_tau=fit_vals['35853']['tau'],
    plant_K=fit_vals['35853']['K'],
    plant_L=fit_vals['35853']['L'],
    Kp=AfracKp,
    Ti=AfracTi,
    Td=0,
    smooth_tau=Afracsmooth_tau
)
fig, ax = plt.subplots(2, 1, sharex=True, gridspec_kw={'hspace': 0.05},
                      figsize=(3.5, 3.475))
ax[0].loglog(ff, np.ones_like(ff), label=None, color='k')
ax[0].loglog(ff, np.abs(plant_TF), label='Fitted Plant')
ax[0].loglog(ff, np.abs(PID_TF), label='PI Controller')
ax[0].loglog(ff, np.abs(OLTF), label='OLTF') # label='Open-loop Transfer Function')
ax[0].loglog(ff, np.abs(CLTF), label='CLTF') # label='Closed-loop Transfer Function')
ax[0].set_ylim(1e-3, 2e3)
ax[0].vlines(UGF, 1e-3, 1e3, color='grey', ls='--')
ax[1].semilogx(ff, np.angle(plant_TF, deg=True), label='Fitted Plant')
ax[1].semilogx(ff, np.angle(PID_TF, deg=True), label='PI Controller')
ax[1].semilogx(ff, np.angle(OLTF, deg=True), label='Open-loop Transfer Function')
ax[1].semilogx(ff, np.angle(CLTF, deg=True), label='Closed-loop Transfer Function')
ax[1].set_ylim(-190, 190)
ax[1].vlines(UGF, -190, 190, color='grey', ls='--')
ax[1].set_yticks([-180, -135, -90, -45, 0, 45, 90, 135, 180])
# ax[1].legend(ncol=2, fontsize=7, loc=(.02, 1.0))
ax[0].legend(ncol=2, loc=(0.35, 0.72))
ax[1].set_xlim(1e-2, 1e1)
ax[1].set_xlabel('Frequency / Hz')
ax[0].set_ylabel('Magnitude')
ax[1].set_ylabel('Phase / ' + r'$^\circ$')
ax[1].yaxis.grid(False, which='minor')

# with open('Afrac_CLTF_values.tex', 'w') as f:
#     f.write(r'\newcommand{\AfracKp}{K$_p$ = ' + f'{AfracKp:.1f}' + '}\n')
#     f.write(r'\newcommand{\AfracTi}{T$_i$ = ' + f'{AfracTi*1e3:.1f}' + ' ms}\n')
#     f.write(r'\newcommand{\Afracstau}{$\tau_s$ = ' + f'{Afracsmooth_tau*1e3:.1f}' + ' ms}\n')
#     f.write(r'\newcommand{\AfracUGF}{' + f'{UGF:.2f}' + ' Hz}\n')
#     f.write(r'\newcommand{\AfracPhaseMargin}{' + f'{phase_margin:.1f}' + ' $^\circ$}\n')
#     f.write(r'\newcommand{\AfracDelayMargin}{' + f'{delay_margin*1e3:.0f}' + ' ms}\n')

# fig.savefig(f'../figures/Afrac_LoopStability.pdf', bbox_inches='tight')

In [ ]:
Kp = -10
Td = 0.0
Ti = 0.253
PID_only = ct.tf([Kp * Td * Ti, Kp * Ti, Kp], [Ti, 0])

PID_only

In [ ]:
smooth_tau = 0.05
PID_presmooth = ct.tf([1 / smooth_tau], [1, 1 / smooth_tau])
PID_presmooth

In [ ]:
PID = ct.series(PID_presmooth, PID_only)
PID

### Surrogate Model SysID

In [ ]:
fit_vals['35854'] = fit_shot('35854', '\\SMSQODMAX', '\\GVSPVBLN2',
                            tlims=(7.5, 14), pre_ext=0,
                            bounds=([-np.inf, 0, 0], [np.inf, 1, 1.1]), pade_order=20,)
fit_vals['35854']

In [ ]:
sn = '35854'
LPz=LPz
lporder = 4
fcut = 20
vmin = 0.05
vmax = 0.25
starttime = 7.5
colormap = cm.plasma

LPFsos = signal.butter(lporder, fcut, 'low', output='sos', fs=1000)

fig, axs = plt.subplots(5, 1, sharex=True, gridspec_kw={'hspace': 0.1, 'height_ratios':[1.0, 1.0, 1.0, 0.5, 0.5]},
                    figsize=(7.5, 4.5))
ax = axs[0]
axSM = axs[1]
axN2 = axs[2]
axafrac = axs[3]
axbeta = axs[4]

with h5py.File(data_filename, 'r') as h5:
    axafrac.plot(*ch(h5, sn, 'PCS_KSTAR', '\\DVSAFRAC'))
    axSM.plot(*ch(h5, sn, 'PCS_KSTAR', '\\SMSQODMAX'), label='Measured')
    axSM.plot(fit_vals[sn]['fit_tt'], fit_vals[sn]['fit_yy'],
                 label='SysID Fit', color='black', ls='--')

    axN2.plot(*clip_flow(h5, sn, 'L', 'N2'))
    caxN2 = axN2.twinx()
    caxN2.plot(*cum_flow(h5, sn, 'L', 'N2'), ls='--', color='tab:brown')

    axbeta.plot(h5[sn]['PCS_KSTAR']['\\EFSBETAN']['dim0'][:],
            h5[sn]['PCS_KSTAR']['\\EFSBETAN']['data'][:],)
    axip = axbeta.twinx()

    axip.plot(h5[sn]['PCS_KSTAR']['\\DVSIP']['dim0'][:],
                -h5[sn]['PCS_KSTAR']['\\DVSIP']['data'][:], ls='--', color='tab:brown')
    
    tt = h5[sn]['PCS_KSTAR']['\\PCLPEP51']['dim0'][:]
    LPdata = np.ones((len(LPz), len(tt))) * np.nan
    for ii in range(len(LPz)):
        LPdata[ii, :] = signal.sosfiltfilt(LPFsos, h5[sn]['PCS_KSTAR']['\\PCLPEP5' + str(ii + 1)]['data'][:])
    
    LPz_arr = np.linspace(min(LPz), max(LPz), 100)
    LPcs = interpolate.CubicSpline(LPz, LPdata, axis=0, bc_type='natural')
    LPplot = ax.pcolor(tt[::20], LPz_arr, LPcs(LPz_arr)[:, ::20], cmap=colormap, shading='nearest', vmin=vmin, vmax=vmax, label=None, edgecolor='face')

    ax.hlines(LPz, color='grey', ls='--', xmin=ax.get_xlim()[0], xmax=ax.get_xlim()[1], alpha=0.8,)
    
    try:
        ax.plot(h5[sn]['EFIT01']['\\ZVSOUT']['dim0'][:],
                h5[sn]['EFIT01']['\\ZVSOUT']['data'][:],
                color='black',
                label='Z' + r'$_{Strike, out}$' + ' from EFIT')
    except BaseException:
        pass
    
    for ii in range(len(tt)):
        if tt[ii] > starttime and  -h5[sn]['PCS_KSTAR']['\\DVSIP']['data'][ii] < 0.4:
            endtime = tt[ii]
            # print(endtime)
            break

fitK = fit_vals[sn]['K'][()]
fitKerr = fit_vals[sn]['perr'][0][()]
fittau = fit_vals[sn]['tau'][()]
fittauerr = fit_vals[sn]['perr'][1][()]
fitL = fit_vals[sn]['L'][()]
fitLerr = fit_vals[sn]['perr'][2][()]
props = dict(boxstyle='round', facecolor='white', alpha=0.8)
axSM.text(14.5, 4.5, f'K = {fitK:0.3f}' + r'$\pm$' + f'{fitKerr:0.3f}\n'
                + r'$\tau$' + f' = {fittau:0.2f}' + r'$\pm$' + f'{fittauerr:0.2f}' + ' s\n'
                + f'L = {fitL:0.3f}' + r'$\pm$' + f'{fitLerr:0.3f} s',
            bbox=props)

# with open('Afrac_SysID_values.tex', 'w') as f:
#     f.write(r'\newcommand{\AfracK}{' + f'K = {fitK:0.3f}' + r'$\pm$' + f'{fitKerr:0.3f}' + '}\n')
#     f.write(r'\newcommand{\AfracTau}{' + r'$\tau$' + f' = {fittau:0.2f}' + r'$\pm$' + f'{fittauerr:0.2f}' + 's}\n')
#     f.write(r'\newcommand{\AfracL}{' + f'L = {fitL:0.3f}' + r'$\pm$' + f'{fitLerr:0.3f}' + 's}\n')
    
caxl = fig.add_axes([0.91, 0.5, 0.015, 0.37])
fig.colorbar(LPplot, cax = caxl, label='LP Array I' + r'$_{sat}$' + ' / A')

ax.set_ylim(min(LPz)-0.002, max(LPz)+0.002)

axSM.set_ylim(4., 7.0)
axafrac.set_ylim(-0., 1.2)
# caxN2.set_ylim(-1, 6)
axbeta.set_ylim(0, 3.0)
axip.set_ylim(0, 0.75)

ax.set_ylabel('Z / m')
axSM.set_ylabel('DivControlNN\nHeat Flux / a.u.')
axafrac.set_ylabel('A' + '$_{frac}$')
axN2.set_ylabel(r'PVBL N$_2$' + '\nCmd / V')
# axN2.set_ylabel(r'PVBL Ne' + '\nCmd / V')
caxN2.set_ylabel('Total Particles\n/1e19', color='tab:brown')
axbeta.set_ylabel(r'$\beta_N$')
axip.set_ylabel('I' + r'$_p$' + ' / MA', color='tab:brown')



axs[-1].set_xlabel('Time / s')

endtime=16.0
axs[-1].set_xlim((starttime, endtime))

ax.legend(
    # loc=(0.6, 0.05), 
    loc='upper right',
    framealpha=0.5)
axSM.legend(ncols=2,
            #    loc=(0.6, 0.75),
               loc='upper right',
)

add_subplot_ind(ax, 0.01, '(a)')
add_subplot_ind(axSM, 0.01, '(b)')
add_subplot_ind(axN2, 0.01, '(c)')
add_subplot_ind(axafrac, 0.01, '(d)')
add_subplot_ind(axbeta, 0.01, '(e)')

ax.grid(False, 'both')
caxN2.yaxis.grid(False, which='minor')
axip.yaxis.grid(False, which='minor')

# fig.savefig(f'../figures/DetCtrl_2D_{sn}.pdf', bbox_inches='tight')

For 36161:

SMKp = -3
SMTi = 0.0685
SMsmooth_tau = 0.005

For 36162:



In [ ]:
SMKp = -3
SMTi=0.0685
SMsmooth_tau=0.005
ff, plant_TF, PID_TF, OLTF, CLTF, Kp, smooth_tau, Ti, Td, UGF, PSF, gain_margin, phase_margin, delay_margin = loop_stability(
    plant_tau=fit_vals['35854']['tau'],
    plant_K=fit_vals['35854']['K'],
    plant_L=0.1, #fit_vals['35853']['L'],
    Kp=SMKp,
    Ti=SMTi,
    Td=0,
    smooth_tau=SMsmooth_tau
)
fig, ax = plt.subplots(2, 1, sharex=True, gridspec_kw={'hspace': 0.05},
                      figsize=(3.5, 3.475))
ax[0].loglog(ff, np.ones_like(ff), label=None, color='k')
ax[0].loglog(ff, np.abs(plant_TF), label='Fitted Plant')
ax[0].loglog(ff, np.abs(PID_TF), label='PI Controller')
ax[0].loglog(ff, np.abs(OLTF), label='OLTF') # label='Open-loop Transfer Function')
ax[0].loglog(ff, np.abs(CLTF), label='CLTF') # label='Closed-loop Transfer Function')
ax[0].set_ylim(1e-3, 2e3)
ax[0].vlines(UGF, 1e-3, 1e3, color='grey', ls='--')
ax[1].semilogx(ff, np.angle(plant_TF, deg=True), label='Fitted Plant')
ax[1].semilogx(ff, np.angle(PID_TF, deg=True), label='PI Controller')
ax[1].semilogx(ff, np.angle(OLTF, deg=True), label='Open-loop Transfer Function')
ax[1].semilogx(ff, np.angle(CLTF, deg=True), label='Closed-loop Transfer Function')
ax[1].set_ylim(-190, 190)
ax[1].vlines(UGF, -190, 190, color='grey', ls='--')
ax[1].set_yticks([-180, -135, -90, -45, 0, 45, 90, 135, 180])
# ax[1].legend(ncol=2, fontsize=7, loc=(.02, 1.0))
ax[0].legend(ncol=2, loc=(0.35, 0.72))
ax[1].set_xlim(1e-2, 1e1)
ax[1].set_xlabel('Frequency / Hz')
ax[0].set_ylabel('Magnitude')
ax[1].set_ylabel('Phase / ' + r'$^\circ$')
ax[1].yaxis.grid(False, which='minor')

# with open('SM_CLTF_values.tex', 'w') as f:
#     f.write(r'\newcommand{\SMKp}{K$_p$ = ' + f'{SMKp:.1f}' + '}\n')
#     f.write(r'\newcommand{\SMTi}{T$_i$ = ' + f'{SMTi*1e3:.1f}' + ' ms}\n')
#     f.write(r'\newcommand{\SMstau}{$\tau_s$ = ' + f'{SMsmooth_tau*1e3:.1f}' + ' ms}\n')
#     f.write(r'\newcommand{\SMUGF}{' + f'{UGF:.2f}' + ' Hz}\n')
#     f.write(r'\newcommand{\SMPhaseMargin}{' + f'{phase_margin:.1f}' + ' $^\circ$}\n')
#     f.write(r'\newcommand{\SMDelayMargin}{' + f'{delay_margin*1e3:.0f}' + ' ms}\n')

fig.savefig(f'../figures/SM_LoopStability.pdf', bbox_inches='tight')

In [ ]:
fig, axs = plt.subplots(2, 2, sharex=True, gridspec_kw={'hspace': 0.05, 'wspace': 0.1},
                        figsize=(7.0, 3.5))

ax = axs[:, 0]
ff, plant_TF, PID_TF, OLTF, CLTF, Kp, smooth_tau, Ti, Td, UGF, PSF, gain_margin, phase_margin, delay_margin = loop_stability(
    plant_tau=fit_vals['35853']['tau'],
    plant_K=fit_vals['35853']['K']/2,
    plant_L=fit_vals['35853']['L'],
    Kp=AfracKp,
    Ti=AfracTi,
    Td=0,
    smooth_tau=Afracsmooth_tau
)
ax[0].loglog(ff, np.ones_like(ff), label=None, color='k')
ax[0].loglog(ff, np.abs(plant_TF), label='Fitted Plant')
ax[0].loglog(ff, np.abs(PID_TF), label='PI Controller')
ax[0].loglog(ff, np.abs(OLTF), label='OLTF') # label='Open-loop Transfer Function')
ax[0].loglog(ff, np.abs(CLTF), label='CLTF') # label='Closed-loop Transfer Function')
ax[0].set_ylim(1e-3, 2e3)
ax[0].vlines(UGF, 1e-3, 1e3, color='grey', ls='--')
ax[1].semilogx(ff, np.angle(plant_TF, deg=True), label='Fitted Plant')
ax[1].semilogx(ff, np.angle(PID_TF, deg=True), label='PI Controller')
ax[1].semilogx(ff, np.angle(OLTF, deg=True), label='Open-loop Transfer Function')
ax[1].semilogx(ff, np.angle(CLTF, deg=True), label='Closed-loop Transfer Function')
ax[1].set_ylim(-190, 190)
ax[1].vlines(UGF, -190, 190, color='grey', ls='--')
ax[1].set_yticks([-180, -135, -90, -45, 0, 45, 90, 135, 180])
# ax[1].legend(ncol=2, fontsize=7, loc=(.02, 1.0))
ax[0].legend(ncol=2, loc=(0.35, 0.72))
ax[1].set_xlim(1e-2, 1e1)
ax[1].set_xlabel('Frequency / Hz')
ax[0].set_ylabel('Magnitude')
ax[1].set_ylabel('Phase / ' + r'$^\circ$')
ax[1].yaxis.grid(False, which='minor')
add_subplot_ind(ax[0], 0.01, '(a)')

ax = axs[:, 1]
ff, plant_TF, PID_TF, OLTF, CLTF, Kp, smooth_tau, Ti, Td, UGF, PSF, gain_margin, phase_margin, delay_margin = loop_stability(
    plant_tau=fit_vals['35854']['tau'],
    plant_K=fit_vals['35854']['K'],
    plant_L=0.1, #fit_vals['35853']['L'],
    Kp=SMKp,
    Ti=SMTi,
    Td=0,
    smooth_tau=SMsmooth_tau
)
ax[0].loglog(ff, np.ones_like(ff), label=None, color='k')
ax[0].loglog(ff, np.abs(plant_TF), label='Fitted Plant')
ax[0].loglog(ff, np.abs(PID_TF), label='PI Controller')
ax[0].loglog(ff, np.abs(OLTF), label='OLTF') # label='Open-loop Transfer Function')
ax[0].loglog(ff, np.abs(CLTF), label='CLTF') # label='Closed-loop Transfer Function')
ax[0].set_ylim(1e-3, 2e3)
ax[0].vlines(UGF, 1e-3, 1e3, color='grey', ls='--')
ax[1].semilogx(ff, np.angle(plant_TF, deg=True), label='Fitted Plant')
ax[1].semilogx(ff, np.angle(PID_TF, deg=True), label='PI Controller')
ax[1].semilogx(ff, np.angle(OLTF, deg=True), label='Open-loop Transfer Function')
ax[1].semilogx(ff, np.angle(CLTF, deg=True), label='Closed-loop Transfer Function')
ax[1].set_ylim(-190, 190)
ax[1].vlines(UGF, -190, 190, color='grey', ls='--')
ax[1].set_yticks([-180, -135, -90, -45, 0, 45, 90, 135, 180])
# ax[1].legend(ncol=2, fontsize=7, loc=(.02, 1.0))
ax[0].legend(ncol=2, loc=(0.35, 0.72))
ax[1].set_xlim(1e-2, 1e1)
ax[1].set_xlabel('Frequency / Hz')
# ax[0].set_ylabel('Magnitude')
# ax[1].set_ylabel('Phase / ' + r'$^\circ$')
ax[0].set_yticklabels([])
ax[1].set_yticklabels([])
ax[1].yaxis.grid(False, which='minor')
add_subplot_ind(ax[0], 0.01, '(b)')

# fig.savefig("../figures/Loop_Stability_Joined.pdf", bbox_inches='tight')

In [ ]:
ax[1].yaxis.get_scale()

## Results

In [ ]:
sn = '35857'
LPz=LPz
lporder = 4
fcut = 20
vmin = 0.05
vmax = 0.25
starttime = 7.5
colormap = cm.plasma

LPFsos = signal.butter(lporder, fcut, 'low', output='sos', fs=1000)

fig, axs = plt.subplots(5, 1, sharex=True, gridspec_kw={'hspace': 0.1, 'height_ratios':[1.0, 1.0, 1.0, 0.5, 0.5]},
                    figsize=(7.5, 4.5))
ax = axs[0]
axafrac = axs[1]
axN2 = axs[2]
axprad = axs[3]
axbeta = axs[4]
with h5py.File(data_filename, 'r') as h5:
    axafrac.plot(*ch(h5, sn, 'PCS_KSTAR', '\\DVSAFRAC'))
    axafrac.plot(*ch(h5, sn, 'PCS_KSTAR','\\DVTAFRAC'),
                 color='black', label='Target', ls='--')

    axN2.plot(*clip_flow(h5, sn, 'L', 'N2'))
    caxN2 = axN2.twinx()
    caxN2.plot(*cum_flow(h5, sn, 'L', 'N2'), ls='--', color='tab:brown')

    axprad.plot(*ch(h5, sn, 'KSTAR', '\\IRVB1_PRAD'))

    axbeta.plot(h5[sn]['PCS_KSTAR']['\\EFSBETAN']['dim0'][:],
            h5[sn]['PCS_KSTAR']['\\EFSBETAN']['data'][:],)
    axip = axbeta.twinx()

    axip.plot(h5[sn]['PCS_KSTAR']['\\DVSIP']['dim0'][:],
                -h5[sn]['PCS_KSTAR']['\\DVSIP']['data'][:], ls='--', color='tab:brown')
    
    tt = h5[sn]['PCS_KSTAR']['\\PCLPEP51']['dim0'][:]
    LPdata = np.ones((len(LPz), len(tt))) * np.nan
    for ii in range(len(LPz)):
        LPdata[ii, :] = signal.sosfiltfilt(LPFsos, h5[sn]['PCS_KSTAR']['\\PCLPEP5' + str(ii + 1)]['data'][:])
    
    LPz_arr = np.linspace(min(LPz), max(LPz), 100)
    LPcs = interpolate.CubicSpline(LPz, LPdata, axis=0, bc_type='natural')
    LPplot = ax.pcolor(tt[::20], LPz_arr, LPcs(LPz_arr)[:, ::20], cmap=colormap, shading='nearest', vmin=vmin, vmax=vmax, label=None, edgecolor='face')

    ax.hlines(LPz, color='grey', ls='--', xmin=ax.get_xlim()[0], xmax=ax.get_xlim()[1], alpha=0.8)

    try:
        ax.plot(h5[sn]['EFIT01']['\\ZVSOUT']['dim0'][:],
                h5[sn]['EFIT01']['\\ZVSOUT']['data'][:],
                color='black',
                label='Z' + r'$_{Strike, out}$' + ' from EFIT')
    except BaseException:
        pass
    
    for ii in range(len(tt)):
        if tt[ii] > starttime and  -h5[sn]['PCS_KSTAR']['\\DVSIP']['data'][ii] < 0.4:
            endtime = tt[ii]
            # print(endtime)
            break

caxl = fig.add_axes([0.91, 0.5, 0.015, 0.37])
fig.colorbar(LPplot, cax = caxl, label='LP Array I' + r'$_{sat}$' + ' / A')

ax.set_ylim(min(LPz)-0.002, max(LPz)+0.002)
axafrac.set_ylim(-0., 1.2)
# caxN2.set_ylim(-1, 6)
axprad.set_ylim(1.8, 4.2)
axbeta.set_ylim(0, 3.0)
axip.set_ylim(0, 0.75)

ax.set_ylabel('Z / m')
axafrac.set_ylabel('A' + '$_{frac}$')
axN2.set_ylabel(r'PVBL N$_2$' + '\nCmd / V')
caxN2.set_ylabel('Total Particles\n/1e19', color='tab:brown')
axprad.set_ylabel('P' + '$_{rad}$\n/ MW')
axbeta.set_ylabel(r'$\beta_N$')
axip.set_ylabel('I' + r'$_p$' + ' / MA', color='tab:brown')

ax.legend(
    # loc=(0.6, 0.05), 
    loc='lower right',
    framealpha=0.5)
axafrac.legend(ncols=2,
            #    loc=(0.6, 0.75),
               loc='upper right',
)

axs[-1].set_xlabel('Time / s')

axs[-1].set_xlim((starttime, endtime))

add_subplot_ind(ax, 0.01, '(a)')
add_subplot_ind(axafrac, 0.01, '(b)')
add_subplot_ind(axN2, 0.01, '(c)')
add_subplot_ind(axprad, 0.01, '(d)')
add_subplot_ind(axbeta, 0.01, '(e)')

ax.grid(False, 'both')
caxN2.yaxis.grid(False, which='minor')
axip.yaxis.grid(False, which='minor')

# fig.savefig(f'../figures/DetCtrl_2D_{sn}.pdf', bbox_inches='tight')

In [ ]:
sn = '36161'
LPz=LPz
lporder = 4
fcut = 20
vmin = 0.05
vmax = 0.25
starttime = 7.0
colormap = cm.plasma

LPFsos = signal.butter(lporder, fcut, 'low', output='sos', fs=1000)

fig, axs = plt.subplots(5, 1, sharex=True, gridspec_kw={'hspace': 0.1, 'height_ratios':[1.0, 1.0, 1.0, 0.5, 0.5]},
                    figsize=(7.5, 4.5))
ax = axs[0]
axSM = axs[1]
axN2 = axs[2]
axafrac = axs[3]
axbeta = axs[4]

with h5py.File(data_filename, 'r') as h5:
    axafrac.plot(*ch(h5, sn, 'PCS_KSTAR', '\\DVSAFRAC'))
    axSM.plot(*ch(h5, sn, 'PCS_KSTAR', '\\SMSQODMAX'), label='Measured')
    axSM.plot(*ch(h5, sn, 'PCS_KSTAR', '\\DVTSMQOD'), label='Target', color='black', ls='--')

    axN2.plot(*clip_flow(h5, sn, 'L', 'N2'))
    caxN2 = axN2.twinx()
    caxN2.plot(*cum_flow(h5, sn, 'L', 'N2'), ls='--', color='tab:brown')

    axbeta.plot(h5[sn]['PCS_KSTAR']['\\EFSBETAN']['dim0'][:],
            h5[sn]['PCS_KSTAR']['\\EFSBETAN']['data'][:],)
    axip = axbeta.twinx()

    axip.plot(h5[sn]['PCS_KSTAR']['\\DVSIP']['dim0'][:],
                -h5[sn]['PCS_KSTAR']['\\DVSIP']['data'][:], ls='--', color='tab:brown')
    
    tt = h5[sn]['PCS_KSTAR']['\\PCLPEP51']['dim0'][:]
    LPdata = np.ones((len(LPz), len(tt))) * np.nan
    for ii in range(len(LPz)):
        LPdata[ii, :] = signal.sosfiltfilt(LPFsos, h5[sn]['PCS_KSTAR']['\\PCLPEP5' + str(ii + 1)]['data'][:])
    
    LPz_arr = np.linspace(min(LPz), max(LPz), 100)
    LPcs = interpolate.CubicSpline(LPz, LPdata, axis=0, bc_type='natural')
    LPplot = ax.pcolor(tt[::20], LPz_arr, LPcs(LPz_arr)[:, ::20], cmap=colormap, shading='nearest', vmin=vmin, vmax=vmax, label=None, edgecolor='face')

    ax.hlines(LPz, color='grey', ls='--', xmin=ax.get_xlim()[0], xmax=ax.get_xlim()[1], alpha=0.8,)
    
    try:
        ax.plot(h5[sn]['EFIT01']['\\ZVSOUT']['dim0'][:],
                h5[sn]['EFIT01']['\\ZVSOUT']['data'][:],
                color='black',
                label='Z' + r'$_{Strike, out}$' + ' from EFIT')
    except BaseException:
        pass
    
    for ii in range(len(tt)):
        if tt[ii] > starttime and  -h5[sn]['PCS_KSTAR']['\\DVSIP']['data'][ii] < 0.4:
            endtime = tt[ii]
            # print(endtime)
            break
    
    axprad = axafrac.twinx()
    axprad.plot(*ch(h5, sn, 'KSTAR', '\\IRVB1_PRAD'), color='tab:brown', ls='--')

caxl = fig.add_axes([0.91, 0.5, 0.015, 0.37])
fig.colorbar(LPplot, cax = caxl, label='LP Array I' + r'$_{sat}$' + ' / A')

ax.set_ylim(min(LPz)-0.002, max(LPz)+0.002)

axSM.set_ylim(4., 11.0)
axafrac.set_ylim(-0., 1.2)
axprad.set_ylim(1.8, 4.2)
# caxN2.set_ylim(-1, 6)
axbeta.set_ylim(0, 3.0)
axip.set_ylim(0, 0.75)

ax.set_ylabel('Z / m')
axSM.set_ylabel('DivControlNN\nHeat Flux / a.u.')
axafrac.set_ylabel('A' + '$_{frac}$')
axprad.set_ylabel('P' + '$_{rad}$\n/ MW', color='tab:brown')
axN2.set_ylabel(r'PVBL N$_2$' + '\nCmd / V')
# axN2.set_ylabel(r'PVBL Ne' + '\nCmd / V')
caxN2.set_ylabel('Total Particles\n/1e19', color='tab:brown')
axbeta.set_ylabel(r'$\beta_N$')
axip.set_ylabel('I' + r'$_p$' + ' / MA', color='tab:brown')



axs[-1].set_xlabel('Time / s')

endtime=16.0
axs[-1].set_xlim((starttime, endtime))

ax.legend(
    # loc=(0.6, 0.05), 
    loc='upper right',
    framealpha=0.5)
axSM.legend(ncols=2,
            #    loc=(0.6, 0.75),
               loc='upper right',
)

add_subplot_ind(ax, 0.01, '(a)')
add_subplot_ind(axSM, 0.01, '(b)')
add_subplot_ind(axN2, 0.01, '(c)')
add_subplot_ind(axafrac, 0.01, '(d)')
add_subplot_ind(axbeta, 0.01, '(e)')

ax.grid(False, 'both')
caxN2.yaxis.grid(False, which='minor')
axip.yaxis.grid(False, which='minor')

# fig.savefig(f'../figures/DetCtrl_2D_{sn}.pdf', bbox_inches='tight')